In [1]:
# Внимание!!! Важно, что бы файлы с данными и исполняемый файл находились в одной папке, 
# тогда пути к тестовым и тренировочным наборам будут содержать только имена файлов.
# 
# В пути к тренировочным и тестовым данным запрежается использовать абсалютную адресацию, 
# то есть адресацию, в которой присутствуют имена папок. Путь должен содержать только имя файла.
#
# Напоминание: под моделью машинного обучения понимаются все действия с исходными данными, 
# которые необходимо произвести, что бы сопоставить признаки целевому значению.

### Область работы 1 (библиотеки)

In [2]:
# Данный блок в области 1 выполняется преподавателем
# 
# данный блок предназначен только для подключения необходимых библиотек
# запрещается подключать библиотеки в других блоках
#
# установка дополнительных библиотек размещается прямо здесь (обязательно закоментированы)
#
# pip install

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import pandas.plotting as pd_plt
import matplotlib.colors as plt_colors
from IPython.display import display, display_html

from sklearn import linear_model
from sklearn.linear_model import Ridge, SGDRegressor, LassoLars , PassiveAggressiveRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.linear_model import QuantileRegressor
from sklearn.linear_model import LinearRegression

from sklearn.pipeline import make_pipeline, Pipeline
import category_encoders as ce
from sklearn.impute import KNNImputer, SimpleImputer
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler, Normalizer, QuantileTransformer, PowerTransformer
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.metrics import classification_report
from sklearn.metrics import make_scorer, mean_squared_error, r2_score
from sklearn.preprocessing import OrdinalEncoder,LabelEncoder, OneHotEncoder, TargetEncoder
from sklearn.compose import ColumnTransformer, make_column_transformer, TransformedTargetRegressor
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import validation_curve
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.inspection import permutation_importance
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from IPython.display import display
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression
from sklearn.ensemble import GradientBoostingRegressor
import shap
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.svm import SVR

### Область работы 2 (выполнение лучшей модели)

In [4]:
# Данный блок(и) в области 2 выполняется преподавателем
#
# В области находится одна, единственная, итоговая модель машинного обучения с однозначными, 
# зафиксированными параметрами
#
# В данной области категорически запрещается искать, выбирать, улучшать, оптимизировать, 
# тюниговать и т.д. модель машинного обучения

In [5]:
# Путь к тренировочному набору
path_train = 'train.csv' # содержит только имя файла, без имен папок
# Путь к тестовому набору
path_test  = 'test.csv' # содержит только имя файла, без имен папок

In [6]:
# Блок обучения модели

In [11]:
X_test = pd.read_csv(path_test)
df = pd.read_csv(path_train)
X = df.drop(['Weight'], axis=1)
y = df['Weight']
Age_map = [{
    'col':'Age',##### Обратить внимание
    'mapping':{'>20': 5,'15-20': 4,'10-15':3,'5-10':2, '2-5':1, '<2':0}
    }]

Age_transformer = Pipeline(steps=[
    ('missing_num', SimpleImputer(strategy = 'most_frequent')),
    ('ce',ce.OrdinalEncoder(mapping=Age_map)),
    ('scaler', MinMaxScaler())
    ])    
sex_transformer = Pipeline(steps=[
    ('missing_num', SimpleImputer(strategy='most_frequent')),
    ('ce',OneHotEncoder(sparse_output=False)),
   # ('scaler', MinMaxScaler())
    ])   
num = ['Length', 'Girth', 'Height']
num_transformer = Pipeline(steps=[
  #  ('missing_num', SimpleImputer(strategy='most_frequent')),
 #   ('ce',OneHotEncoder(sparse_output=False)),
    ('polynom', PolynomialFeatures(3,include_bias=False, interaction_only = True)),
    ('scaler', MinMaxScaler())
    ])  
bcs = ['BCS']
bcs_transformer = Pipeline(steps=[
  #  ('missing_num', SimpleImputer(strategy='most_frequent')),
 #   ('ce',OneHotEncoder(sparse_output=False)),
 #   ('polynom', PolynomialFeatures(3,include_bias=False, interaction_only = True)),
    ('scaler', StandardScaler())
    ])  
CT = ColumnTransformer([
        ("Age", Age_transformer, ['Age']),# убрать быстую разряженную 
        ("Sex", sex_transformer, ['Sex']),
        ("num", num_transformer, num),
        ("bcs", bcs_transformer, bcs),    
        ]).set_output(transform='pandas')

display(CT)

ct = CT.fit_transform(X)
pd.DataFrame(ct).head().T

ColumnTransformer(transformers=[('Age',
                                 Pipeline(steps=[('missing_num',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('ce',
                                                  OrdinalEncoder(mapping=[{'col': 'Age',
                                                                           'data_type': dtype('O'),
                                                                           'mapping': >20      5
15-20    4
10-15    3
5-10     2
2-5      1
<2       0
dtype: int64}])),
                                                 ('scaler', MinMaxScaler())]),
                                 ['Age']),
                                ('Sex',
                                 Pipeline(steps=[('missing_num',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('ce',
                                                  OneHotEncoder(sparse_output=False))]),
                                 ['Sex']),
                                ('num',
                                 Pipeline(steps=[('polynom',
                                                  PolynomialFeatures(degree=3,
                                                                     include_bias=False,
                                                                     interaction_only=True)),
                                                 ('scaler', MinMaxScaler())]),
                                 ['Length', 'Girth', 'Height']),
                                ('bcs',
                                 Pipeline(steps=[('scaler', StandardScaler())]),
                                 ['BCS'])])

,0,1,2,3,4
Age__Age,0.400000,0.400000,0.600000,0.200000,0.000000
Sex__Sex_female,0.000000,1.000000,0.000000,1.000000,1.000000
Sex__Sex_gelding,0.000000,0.000000,1.000000,0.000000,0.000000
Sex__Sex_stallion,1.000000,0.000000,0.000000,0.000000,0.000000
num__Length,0.803030,0.681818,0.803030,0.696970,0.606061
num__Girth,0.705882,0.838235,0.838235,0.573529,0.573529
num__Height,0.690476,0.619048,0.833333,0.595238,0.500000
num__Length Girth,0.710227,0.702221,0.786932,0.570248,0.516012
num__Length Height,0.732634,0.614136,0.798233,0.614688,0.513087
num__Girth Height,0.667794,0.720609,0.830714,0.536503,0.494728


In [12]:
Rridge = Pipeline(steps=[
    ('preproc', CT),
    ('estimator', Ridge(alpha=0.1)) 
])
Rridge.fit(X, y)


Pipeline(steps=[('preproc',
                 ColumnTransformer(transformers=[('Age',
                                                  Pipeline(steps=[('missing_num',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('ce',
                                                                   OrdinalEncoder(mapping=[{'col': 'Age',
                                                                                            'data_type': dtype('O'),
                                                                                            'mapping': >20      5
15-20    4
10-15    3
5-10     2
2-5      1
<2       0
dtype: int64}])),
                                                                  ('scaler',
                                                                   MinMaxScaler())]),
                                                  ['Age']),
                                                 ('Sex',
                                                  Pipeline(steps=[('missing_num',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('ce',
                                                                   OneHotEncoder(sparse_output=False))]),
                                                  ['Sex']),
                                                 ('num',
                                                  Pipeline(steps=[('polynom',
                                                                   PolynomialFeatures(degree=3,
                                                                                      include_bias=False,
                                                                                      interaction_only=True)),
                                                                  ('scaler',
                                                                   MinMaxScaler())]),
                                                  ['Length', 'Girth',
                                                   'Height']),
                                                 ('bcs',
                                                  Pipeline(steps=[('scaler',
                                                                   StandardScaler())]),
                                                  ['BCS'])])),
                ('estimator', Ridge(alpha=0.1))])

In [13]:
# Блок предсказания с использованием тестового набора

In [14]:
y_predict = Rridge.predict(X_test)

In [15]:
# Вектора предсказанных значений  y_predict полученый на основане тестового набора
y_predict

array([145.16857329, 143.36409901, 162.06127866, 156.98878556,
       163.59245899, 125.05344273, 165.0225365 , 153.08323816,
       140.91821314, 174.71079863, 130.77037542, 152.50367912,
       198.12132162, 134.36960158, 123.81887813, 171.36945773,
       160.56069718, 149.31919707, 147.11910244, 141.9977502 ,
       159.40518079, 157.2761755 , 143.56231062, 156.79436647,
       174.33852294, 171.41122445, 153.75646348,  88.66538801,
       158.00051116, 160.89826309, 158.93103067, 154.43586335,
       194.36135691, 195.57899042, 127.4909918 , 163.48200383,
       169.73369109, 178.61635374, 116.41846124, 178.84910962,
       159.89135565, 154.53693211, 172.52128139, 147.30019817,
       162.09878337, 168.5951365 , 138.83862169, 108.20525435,
       202.68050876, 153.04435667, 154.06159935, 152.21438573,
       181.47681799, 159.69690656, 183.67767453, 174.54137234,
       121.2371667 , 163.05733685, 147.1917102 , 139.60083937,
       178.23842401, 100.76204462, 133.92050611, 166.05